In [ ]:
## Installation
%pip install fastapi uvicorn nest-asyncio   # API
%pip install numpy                          # Efficient data handling
%pip install python-dotenv                  # .env => extracting hidden info
%pip install requests                       # Performing API calls
% pip install statsmodels                   # Optimized Parameter Generation

## Imports
from fastapi import FastAPI, Cookie, HTTPException
from fastapi.responses import JSONResponse
from fastapi.middleware.cors import CORSMiddleware
from typing import Annotated
import uvicorn
import nest_asyncio
import numpy as np
from dotenv import load_dotenv
import os
import requests
from enum import Enum 

In [ ]:
## Configuration
load_dotenv()

## Global Variables
BACKEND_URL = os.getenv("BACKEND_URL")

In [ ]:
app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["http://localhost:5173", "http://localhost:3000"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"]
)

In [ ]:
# SARIMA Parameters
# TODO: Currently speculation, fix down the line
p = 1       # 1st Order Non-Seasonal AR Component
d = 1       # 1st Order Non-Seasonal I Component
q = 1       # 1st Order Non-Seasonal MA Component
P = 1       # 1st Order Seasonal AR Component
D = 1       # 1st Order Seasonal I Component
Q = 1       # 1st Order Seasonal MA Component
s = 52      # Weekly
 

In [ ]:
# TODO: Put data (num orders, total revenue, num first-time orders, num site visits) for a given restaurant into buckets


In [ ]:
# Apply differencing
# Credit: https://otexts.com/fpp2/stationarity.html
def differencing(arr, lag = 1, repeat = 1):
    differenced_arr = np.copy(arr)
    for i in range(repeat):
        differenced_arr = differenced_arr[lag:] - differenced_arr[:lag]
    return differenced_arr

# Apply undifferencing
# Credit: https://stackoverflow.com/questions/72700812/how-to-inverse-differencing-on-future-forecasted-result
def revert_differencing(original_arr, differenced_arr, lag = 1, repeat = 1):
    inverted_diff_arr = np.copy(differenced_arr)
    for i in range(repeat):
        inverted_diff_arr = np.r_[original_arr[-lag:], differenced_arr].cumsum()
    return inverted_diff_arr[-len(differenced_arr):]

In [ ]:
# Perform polynomial multiplication (essentially performing discrete convolution)
def polynomial_mul(poly_a, poly_b):
    product_len = len(poly_a) + len(poly_b) - 1
    product_poly = np.zeros(product_len, dtype=float)
    for i in range(len(poly_a)):
        for j in range(len(poly_b)):
            product_poly[i + j] += poly_a[i] * poly_b[j]     
    return product_poly

# Using seasonal and non-seasonal coefficients, generates the polynomial upon application of both seasonal/non-seasonal for one of: {MA, AR}
def generate_seasonal_n_nonseasonal_poly(seasonal_coeff, nonseasonal_coeff, is_positive):
    # TODO: Use s from above
    # Nonseasonal Polynomial w/ Lag Coefficients
    nonseasonal_poly = np.zeros(len(nonseasonal_coeff) + 1) 
    nonseasonal_poly[0] = 1
    nonseasonal_poly[1:] = np.array(nonseasonal_coeff) if is_positive else -np.array(nonseasonal_coeff)
    
    # Seasonal Polynomial w/ Lag Coefficients
    seasonal_poly = np.zeros((len(seasonal_coeff) * s) + 1)
    seasonal_poly[0] = 1
    for i in range(s):
        seasonal_poly[s*i + 1] = seasonal_coeff[i] if is_positive else -seasonal_coeff[i]
        
    # Generate Product Polynomial
    product_poly = polynomial_mul(nonseasonal_poly, seasonal_poly)
    
    return product_poly

In [ ]:
# Perform parameter generation from scratch
# NOTE: Implementing this from scratch would require using Hannan–Rissanen and Maximum Likelihood Estimation (MLE) with a Kalman filter and additional optimizers, which would
# add an extreme and highly unrealistic level of complexity (due to the very difficult math involved)
# NOTE: This ONLY uses model parameters - everything else is being done entirely from scratch

def get_sarima_params(data):
    from statsmodels.tsa.statespace.sarimax import SARIMAX
    model = SARIMAX(data, order=(p,d,q), seasonal_order=(P,D,Q,s))
    res = model.fit(disp=False)
    params = res.params
    
    phi1   = params[0]
    theta1 = params[1]
    Phi1   = params[2]
    Theta1 = params[3]
    sigma = params[-1]

    return dict(phi1=phi1, theta1=theta1, Phi1=Phi1, Theta1=Theta1, sigma=sigma)

In [ ]:
# TODO: 
def forecast_next_week(data):
    # Retrieve SARIMA parameters
    params = get_sarima_params(data)
    phi1, Phi1 = params['phi1'], params['Phi1']
    theta1, Theta1 = params['theta1'], params['Theta1']
    sigma = params['sigma']
    
    # Apply seasonal differencing
    differenced_trend = differencing(data, s)
    # Apply trend differencing on top of this
    differenced_all = differencing(differenced_trend, 1)
    
    # TODO: 

In [ ]:
# NOTE: Given limited data, will only be used to forecast next week's KPI figures
@app.get("/analytics/forecast")
def forecast():
    # TODO: Get data
    next_week_revenue = forecast_next_week(revenue_data)
    next_week_orders = forecast_next_week(order_data)
    next_week_site_visits = forecast_next_week(visit_data)
    next_week_first_time_consumers = forecast_next_week(first_time_consumer_data)
    
    return JSONResponse([next_week_revenue, next_week_orders, next_week_site_visits, next_week_first_time_consumers])

In [ ]:
## Get FastAPI Server Online
if __name__ == "__main__":
    nest_asyncio.apply()
    uvicorn.run(app, host="127.0.0.1", port=8080)